# PurpleAir Stations QAQC

## Import Packages

In [2]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times

# Database 

# import psycopg2
# from psycopg2 import sql

# Analysis

import numpy as np
import geopandas as gpd
import pandas as pd

# Get CWD

cwd = os.getcwd()

## Definitions

In [3]:
# This is my personal API key... Please use responsibly! 51592903-B445-11ED-B6F4-42010A800007

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


In [5]:
#Setting lat/long for PurpleAir API Parameters

datapath = os.path.join(cwd, '..', '..', 'Data')

extent = gpd.read_file(os.path.join(datapath, 'extent.geojson'))

nwlng, selat, selng, nwlat = extent.to_crs('EPSG:4326').total_bounds

## Importing PurpleAir Station Data from PurpleAir API

In [6]:
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

In [7]:
#Set bounding strings for API parameters
bounds_strings = [f'nwlng={nwlng}',
                  f'nwlat={nwlat}',
                  f'selng={selng}',
                  f'selat={selat}']

bounds_string = '&'.join(bounds_strings)

print(bounds_string)

nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [8]:
#Setting parameters for API
fields = ['name', 'firmware_version','date_created','last_modified','last_seen','uptime','position_rating','channel_state','channel_flags','altitude',
          'location_type','latitude', 'longitude']

fields_string = 'fields=' + '%2C'.join(fields)

print(fields_string)

fields=name%2Cfirmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude


In [9]:
#finalizing query for API function
query_string = '&'.join([fields_string, bounds_string])

print(query_string)

fields=name%2Cfirmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude&nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [10]:
#calling the API
response = getSensorsData(query_string, api)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=name%2Cfirmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude&nwlng=-93.43046973986235&nwlat=45.12326140727048&selng=-93.09304872066019&selat=44.81857616148092


In [11]:
response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

df = pd.DataFrame(data, columns = col_names)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sensor_index      92 non-null     object
 1   last_modified     92 non-null     object
 2   date_created      92 non-null     object
 3   last_seen         92 non-null     object
 4   name              92 non-null     object
 5   location_type     92 non-null     object
 6   firmware_version  92 non-null     object
 7   uptime            92 non-null     object
 8   position_rating   92 non-null     object
 9   latitude          92 non-null     object
 10  longitude         92 non-null     object
 11  altitude          92 non-null     object
 12  channel_state     92 non-null     object
 13  channel_flags     92 non-null     object
dtypes: object(14)
memory usage: 10.2+ KB


### Compare with the list of Ids from City

In [34]:
# Load the list the City provided

sensor_info = pd.read_excel(os.path.join(datapath, 'PA IDs and indexes.xlsx')) # Load as DataFrame

ids_from_city = sensor_info['Sensor Index'].dropna().astype(int)

In [35]:
# From what we just queried from PurpleAir 
# Only want the City's monitors

is_city = df_stations.name.apply(lambda x: 'CITY OF MINNEAPOLIS' in x.upper())

ids_from_PurpleAir = df[is_city].sensor_index.astype(int)

In [36]:
# Now let's get the union of these two lists of ids

ids_from_either = np.union1d(ids_from_city.values, ids_from_PurpleAir)

In [37]:
len(ids_from_either)

66

### Final PurpleAir Query

In [45]:
# One final query for this list of ids

# Get start Times

sensor_string = 'show_only=' + '%2C'.join(ids_from_either.astype(str))

query_string = '&'.join([fields_string, sensor_string])

In [46]:
response = getSensorsData(query_string, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

Here is the full url for the API call:

 https://api.purpleair.com/v1/sensors?fields=name%2Cfirmware_version%2Cdate_created%2Clast_modified%2Clast_seen%2Cuptime%2Cposition_rating%2Cchannel_state%2Cchannel_flags%2Caltitude%2Clocation_type%2Clatitude%2Clongitude&show_only=142718%2C142720%2C142724%2C142726%2C142728%2C142730%2C142732%2C142734%2C142736%2C142744%2C142748%2C142750%2C142752%2C142756%2C142772%2C142774%2C142852%2C143214%2C143216%2C143222%2C143224%2C143226%2C143238%2C143240%2C143242%2C143246%2C143248%2C143634%2C143636%2C143648%2C143656%2C143660%2C143666%2C143668%2C143916%2C143942%2C143944%2C145202%2C145204%2C145234%2C145242%2C145250%2C145262%2C145454%2C145470%2C145498%2C145502%2C145504%2C145506%2C145604%2C145610%2C145614%2C145616%2C156605%2C157747%2C157757%2C157785%2C157787%2C157837%2C157845%2C157861%2C157871%2C157877%2C157935%2C166459%2C168327


## Cleaning PurpleAir Station Data

In [47]:
#visualizing API response
sensors_df.head()

sensor_index last_modified date_created   last_seen  \
0       142718    1675359061   1642013869  1687806688   
1       142720    1675359105   1642013875  1687806659   
2       142726    1675359066   1642013897  1687806636   
3       142724    1681913464   1642013889  1687204606   
4       142730    1675359088   1642013916  1687806654   

                                                name location_type  \
0  City of Minneapolis Community Air Monitoring P...             0   
1  City of Minneapolis community air monitoring p...             0   
2  City of Minneapolis Community Air Monitoring P...             0   
3  City of Minneapolis Community Air Monitoring P...             0   
4  City of Minneapolis Community Air Monitoring P...             0   

  firmware_version uptime position_rating   latitude   longitude altitude  \
0             7.02  69503               5  44.995792  -93.295395      865   
1             7.02  52522               5   44.95617   -93.25471      856   
2             7.02  35933               5   45.01507   -93.28903      889   
3             7.02  40667               5  44.937218  -93.243866      853   
4             7.02  13889               5   44.99218   -93.29627      862   

  channel_state channel_flags  
0             3             0  
1             3             0  
2             3             0  
3             3             0  
4             3             0

In [48]:
#find outside sensors
df_outside = sensors_df[sensors_df.location_type =='0']
len(df_outside)

63

In [49]:
#drop the location_type now that we have filtered for outdoor sensors only
df_stations = df_outside.drop('location_type', axis=1)

In [51]:
#Converting UNIX date/time to pd
df_stations['last_modified'] = pd.to_datetime(df_stations['last_modified'], unit='s')
df_stations['date_created'] = pd.to_datetime(df_stations['date_created'], unit='s')
df_stations['last_seen'] = pd.to_datetime(df_stations['last_seen'], unit='s')

In [52]:
df_stations

sensor_index       last_modified        date_created           last_seen  \
0        142718 2023-02-02 17:31:01 2022-01-12 18:57:49 2023-06-26 19:11:28   
1        142720 2023-02-02 17:31:45 2022-01-12 18:57:55 2023-06-26 19:10:59   
2        142726 2023-02-02 17:31:06 2022-01-12 18:58:17 2023-06-26 19:10:36   
3        142724 2023-04-19 14:11:04 2022-01-12 18:58:09 2023-06-19 19:56:46   
4        142730 2023-02-02 17:31:28 2022-01-12 18:58:36 2023-06-26 19:10:54   
..          ...                 ...                 ...                 ...   
58       157871 2023-02-02 17:28:58 2022-08-03 00:51:31 2023-06-26 19:10:35   
59       157877 2023-02-02 17:28:29 2022-08-03 00:51:38 2023-06-26 19:10:31   
60       157935 2023-02-02 17:29:02 2022-08-03 01:22:05 2023-06-26 19:09:40   
61       166459 2023-04-05 15:37:02 2022-10-11 19:35:58 2023-06-26 19:11:43   
62       168327 2023-02-02 17:32:31 2022-10-24 19:48:08 2023-06-26 19:11:19   

                                                 name firmware_version uptime  \
0   City of Minneapolis Community Air Monitoring P...             7.02  69503   
1   City of Minneapolis community air monitoring p...             7.02  52522   
2   City of Minneapolis Community Air Monitoring P...             7.02  35933   
3   City of Minneapolis Community Air Monitoring P...             7.02  40667   
4   City of Minneapolis Community Air Monitoring P...             7.02  13889   
..                                                ...              ...    ...   
58  City of Minneapolis Community Air Monitoring P...             7.02  29814   
59  City of Minneapolis Community Air Monitoring P...             7.02    334   
60  City of Minneapolis Community Air Monitoring P...             7.02  33704   
61  City of Minneapolis Community Air Monitoring P...             7.02  13355   
62  City of Minneapolis Community Air Monitoring P...             7.02  22722   

   position_rating   latitude   longitude altitude channel_state channel_flags  
0                5  44.995792  -93.295395      865             3             0  
1                5   44.95617   -93.25471      856             3             0  
2                5   45.01507   -93.28903      889             3             0  
3                5  44.937218  -93.243866      853             3             0  
4                5   44.99218   -93.29627      862             3             0  
..             ...        ...         ...      ...           ...           ...  
58               5  44.934963   -93.27042      865             3             2  
59               5  44.900864   -93.20838      849             3             0  
60               5   44.92983   -93.32441      900             3             0  
61               5   44.89575   -93.26829      843             3             0  
62               5   44.95249   -93.26352      872             3             0  

[63 rows x 13 columns]

In [54]:
# Save as geojson

gdf_stations = gpd.GeoDataFrame(df_stations, 
                                geometry = gpd.points_from_xy(
                                    df_stations.longitude,
                                    df_stations.latitude,
                                    crs = 'EPSG:4326')
                               ).to_crs('EPSG:26915')

gdf_stations.to_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

## Insert Data into SQL Table (Not Done)

In [64]:
# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

connection = psycopg2.connect(**pg_connection_dict)

0

In [66]:
#connect to the cursor
cur = connection.cursor()

# iterate over the dataframe and insert each row into the database using a SQL INSERT statement

### NEED TO ADD WKT

for index, row in df_stations.iterrows():
    cur.execute('''
    INSERT INTO PURPLEAIR_STATIONS (sensor_index, last_modified, date_created, last_seen, name, firmware_version, uptime, 
                        position_rating, latitude, longitude, altitude, channel_state, channel_flags, WKT)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326)::geometry)
    ''', (row['sensor_index'], row['last_modified'], row['date_created'], row['last_seen'], row['name'], row['firmware_version'], 
          row['uptime'], row['position_rating'], row['altitude'], 
          row['channel_state'], row['channel_flags'], row['WKT']))
    connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()